In [13]:
import numpy as np
import cv2 
import os
import imutils
import matplotlib.pyplot as plt
from PIL import Image
"""
This function is used to crop the images contained in the path directory and save it in the dest directory
It also displays the cropped images
"""
def crop(path,dest):

    if os.path.exists(dest):
        os.system('rm -r dest')
    if not os.path.exists(dest):
        os.makedirs(dest)
    if not os.path.exists(path):
        print("The given directory doesn't exist")
        return 
    process(path, dest)
    #show_images(dest)
    return

"""
This function is used to display the images contained in the directory
"""
def show_images(directory):
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            if f.endswith('.ppm') or f.endswith('.jpeg') or f.endswith('.jp2'):
                cv2.imshow('image', cv2.imread(f))
                cv2.waitKey()
    return

"""
This function is used to process the images contained in the path directory and save it in the dest directory
"""
def process(path, dest):

    directory = path
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if not os.path.isfile(f):
            print("Expected the directory to contain only files") 
        if os.path.isfile(f):
            if filename.endswith('.ppm') or filename.endswith('.jpeg') or filename.endswith('.jp2'):

    #----------------------------------------------------------------------------------------------------------

                raw_image = cv2.imread(f) # Charge l'image
                process_image = raw_image.copy() # Copie l'image
                # Recherche de contour basé sur la forme et passe à l'image suivante si un contour est trouvé
                if(process_shape(raw_image, dest, filename)):
                    continue
    
    #----------------------------------------------------------------------------------------------------------

                # Floute avec du gaussian puis du median blur
                blur = cv2.GaussianBlur(process_image, (9,9), 0)
                blur = cv2.medianBlur(blur, 9)          

                # Transforme l'image en image HSV
                img_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

                # Augmente la saturation
                img_hsv[...,1] = img_hsv[...,1]*1.4

    #----------------------------------------------------------------------------------------------------------
                # Crée des masques de couleur pour les différents couleur des panneaux routiers(rouge, bleu et orange)

                # Masque rouge pour les valeurs basses en HSV
                lower_red = np.array([0, 50, 50])
                upper_red = np.array([10, 255, 255])
                mask_temp = cv2.inRange(img_hsv, lower_red, upper_red)
                # Masque rouge valeurs hautes
                lower_red = np.array([170, 70, 50])
                upper_red = np.array([180, 255, 255])
                mask_red = cv2.inRange(img_hsv.copy(), lower_red, upper_red)
                # Combine les valeurs basses et hautes du rouge
                mask_red = cv2.bitwise_or(mask_temp, mask_red)

                # Masque orange
                lower_orange = np.array([5, 150, 150])
                upper_orange = np.array([20, 250, 255])
                mask_orange = cv2.inRange(img_hsv.copy(), lower_orange, upper_orange)

                # Masque bleu
                lower_blue = np.array([90, 100, 100])
                upper_blue = np.array([121, 255, 255])
                mask_blue = cv2.inRange(img_hsv.copy(), lower_blue, upper_blue)
    
    #----------------------------------------------------------------------------------------------------------

                # Trouver les contours dans les différents filtres de couleur
                contours_orange, _ = cv2.findContours(mask_orange, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                contours_blue, _ = cv2.findContours(mask_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                contours = contours_orange + contours_red + contours_blue

    #----------------------------------------------------------------------------------------------------------

                # Initialise une liste vide pour stocker les contours valides
                list_contour = []
                best_contour = None

                # Parcourt tous les contours trouvés
                for cnt in contours:
                    # Calcule le rectangle englobant le contour (bounding box)
                    x, y, w, h = cv2.boundingRect(cnt)

                    # Impose que la bounding box autour du contour soit plus grand que 5% de l'image 
                    if (w*h) < (0.001*raw_image.shape[0]*raw_image.shape[1]):
                        continue

                    # Impose que la bounding box ne soit pas un rectangle très plat
                    if (w/h) < 0.7 or (w/h) > 1.3:
                        continue

                    # Si le contour est un cercle, alors c'est très probablement un panneau
                    area = cv2.contourArea(cnt)
                    perimeter = cv2.arcLength(cnt, True)
                    try:
                        circularity = 4 * np.pi * area / perimeter ** 2
                        if circularity > 0.8:
                            best_contour = cnt
                            list_contour.append(cnt)
                            break
                    except: 
                        pass 
                    # On regarde si l'aire du plus petit contour convexe autour du contour est proche de l'aire du contour
                    try:
                        hull = cv2.convexHull(cnt)
                        hull_area = cv2.contourArea(hull)
                        solidity = float(area)/hull_area
                        if solidity < 0.7:
                            continue
                    except: 
                        continue
                    # Si le contour correspond aux critères, on le garde
                    list_contour.append(cnt)

                #----------------------------------------------------------------------------------------------------------
                # Si aucun contour correspondant à nos critères n'a été trouvé
                if(len(list_contour) == 0):  
                    cv2.imwrite(dest+"/"+filename, raw_image)
                    continue
                    
                # Si aucun cerle n'a été trouvé, on prends le contour répondant au critères ayant la plus grande aire
                if(best_contour is None):
                    #get max of list_contour
                    best_contour = max(list_contour, key=lambda x: cv2.contourArea(x))

                #----------------------------------------------------------------------------------------------------------

                # Crop l'image sur la bounding box autour du contour choisi
                (x,y,w,h) = cv2.boundingRect(best_contour)
                if(x-int(w/10) > 0 and y-int(h/10) > 0 and x + w+int(w/10) < raw_image.shape[1] and y + h+int(h/10) < raw_image.shape[0] ):
                    raw_image = raw_image[y-int(h/10):y + h+int(h/10), x-int(w/10):x + w+int(w/10)]
                    # Ecrit l'image croppée dans un dossier
                    cv2.imwrite(dest+"/"+filename, raw_image)
                else: 
                    raw_image = raw_image[y:y + h, x:x + w]
                    # Ecrit l'image croppée dans un dossier
                    cv2.imwrite(dest+"/"+filename, raw_image)
            

    return 


def process_shape(raw_image, dest, filename):

    process_image = raw_image.copy()
    # Transforme l'image afin de détecter les contours
    
    # Converti l'image en HSV
    hsv = cv2.cvtColor(process_image, cv2.COLOR_BGR2HSV)
    # Retire la saturation et couleurs trop sombres
    lower_sat = np.array([50, 50, 50])
    upper_sat = np.array([255, 255, 255])
    mask = cv2.inRange(hsv, lower_sat, upper_sat)
    # Applique le masque pour retirer la saturation
    masked_image = cv2.bitwise_and(process_image, process_image, mask=mask)

    #----------------------------------------------------------------------------------------------------------

    # Aplication d'un filtre gris
    gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    # Application d'un floutage
    bilateral_filtered_image = cv2.GaussianBlur(gray,(3,3),0)
    # Détection des changements de couleurs sur l'images
    edge_detected_image = cv2.Canny(bilateral_filtered_image, 90, 170)

    #----------------------------------------------------------------------------------------------------------

    # Trouve les contours dans l'image et les trier par aire et garder les 20 plus grands
    contours = cv2.findContours(edge_detected_image.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:20]

    shapes = []
    # Itére sur chaque contour
    for contour in contours:

        length = cv2.arcLength(contour,True)
        approx = cv2.approxPolyDP(contour,0.01*length, True)
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if(perimeter == 0):
            continue
        circularity = 4 * np.pi * area / perimeter ** 2
        if circularity < 0.8:
            continue
        # Regarde si le l'approximation du contour est convexe
        if not cv2.isContourConvex(approx):
            continue
        # Si l'approximation a 3 côtés, ça doit être un panneau triangulaire
        if len(approx) == 3:
            shapes.append(contour)
        # Si l'approximation a 4 côtés, on vérifie que le rapport longueur sur largeur est proche de 1
        elif len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            if (ar >= 0.8 and ar <= 1.2):
                shapes.append(contour)
            else:
                continue
        # Si l'approximation a 8 côtés, ça doit être un panneau STOP octogonal
        elif len(approx) == 8:
            shapes.append(contour)
        # Sinon, si le rapport entre l'aire du contour et l'aire du plus petit cercle entourant le contour est proche de 1
        elif len(approx) > 8:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            area = cv2.contourArea(contour)
            if radius > 0 and area / (np.pi * radius**2) >= 0.8:
                shapes.append(contour)
        # Sinon passe au contour suivant
        else:
            continue

    #----------------------------------------------------------------------------------------------------------

    # Garde le contour avec la plus grande aire
    try:
        best_contour = max(shapes, key=lambda x: cv2.contourArea(x))
    except ValueError:
        best_contour = None
    if best_contour is None:
        # Aucun contour correct trouvé
        return False
    
    # Crop l'image sur la bounding box autour du contour choisi
    (x,y,w,h) = cv2.boundingRect(best_contour)
    if(x-int(w/10) > 0 and y-int(h/10) > 0 and x + w+int(w/10) < raw_image.shape[1] and y + h+int(h/10) < raw_image.shape[0] ):
        raw_image = raw_image[y-int(h/10):y + h+int(h/10), x-int(w/10):x + w+int(w/10)]
        # Ecrit l'image croppée dans un dossier
        cv2.imwrite(dest+"/"+filename, raw_image)
    else: 
        raw_image = raw_image[y:y + h, x:x + w]
        # Ecrit l'image croppée dans un dossier
        cv2.imwrite(dest+"/"+filename, raw_image)
    return True
    
def main():
    crop('challenge-2/eval_kaggle2', 'result_cropped_eval2')
    return
if __name__ == "__main__":
    main()

Shape detected
file:  16.ppm
Shape detected
file:  42.ppm
Shape detected
file:  45.ppm
Shape detected
file:  8.ppm
